In [ ]:
!pip install gdown underthesea transformers pyvi nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 13.8 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.WARNING)

# Importing necessary libraries
import os
import sys
import json
import logging
import math
import copy
import re
import torch
from torch import nn

# Data handling libraries
import pandas as pd
import numpy as np

# NLP libraries
from nltk.tokenize import sent_tokenize as nltk_sent_tokenize
import nltk
from underthesea import sent_tokenize as under_sent_tokenize
from underthesea import text_normalize
nltk.download('punkt')

# Multithreading libraries
from multiprocessing import Pool, cpu_count

# Progress bar library
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler

# Google Drive download library
import gdown

No module named 'fasttext'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Evidence Retrieval

In [ ]:
!gdown '1mNalWH4TZFhAvNEIv_RcWQS3RuvNSo52'
!gdown '1_hHef3PtVl8QGv2J1JgIpA726NhBv5GE'
!gdown '1XxKLEOq4XsMshcDzz2ZMAQhrMhohN0av'

import pandas as pd
import ast

train_data = pd.read_csv("/content/train_data.csv")
test_data = pd.read_csv("/content/test_data.csv")
dev_data = pd.read_csv("/content/dev_data.csv")

train_data = train_data.drop('evidence_top5', axis=1)
dev_data = dev_data.drop('evidence_top5', axis=1)
test_data = test_data.drop('evidence_top5', axis=1)


def convert_to_list(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return value

train_data['splited_sentences'] = train_data['splited_sentences'].apply(convert_to_list)
train_data['Evidence_List'] = train_data['Evidence_List'].apply(convert_to_list)
test_data['splited_sentences'] = test_data['splited_sentences'].apply(convert_to_list)
test_data['Evidence_List'] = test_data['Evidence_List'].apply(convert_to_list)
dev_data['splited_sentences'] = dev_data['splited_sentences'].apply(convert_to_list)
dev_data['Evidence_List'] = dev_data['Evidence_List'].apply(convert_to_list)

Downloading...
From: https://drive.google.com/uc?id=1mNalWH4TZFhAvNEIv_RcWQS3RuvNSo52
To: /content/dev_data.csv
100% 7.39M/7.39M [00:00<00:00, 93.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_hHef3PtVl8QGv2J1JgIpA726NhBv5GE
To: /content/test_data.csv
100% 15.1M/15.1M [00:00<00:00, 29.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XxKLEOq4XsMshcDzz2ZMAQhrMhohN0av
To: /content/train_data.csv
100% 53.2M/53.2M [00:04<00:00, 12.8MB/s]


## SBERT

In [ ]:
# Transformers library
import transformers
from transformers import AutoModel, AutoTokenizer
transformers.logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
tokenizer_sbert = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
model_sbert = AutoModel.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1').to(device)

cuda


tokenizer_config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def similarities(context: list, text: str):
    sentences = [text] + context

    encoded_input = tokenizer_sbert(sentences, padding=True, truncation=True, return_tensors='pt')
    encoded_input = {key: value.to('cuda') for key, value in encoded_input.items()}

    # Compute token embeddings
    with torch.no_grad():
        model_output = model_sbert(**encoded_input)
    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    similarities = []
    claim_embeddings = sentence_embeddings[0].unsqueeze(0)
    for i in (range(1, len(sentence_embeddings))):
        evidence_embeddings = sentence_embeddings[i].unsqueeze(0)
        cosine = nn.CosineSimilarity(dim=1, eps=1e-6)
        similarity = cosine(claim_embeddings.to(device), evidence_embeddings.to(device)).item()
        # scaled_similarity = ((similarity + 1) / 2) * weight[i-1]
        similarities.append((sentences[i], similarity))

    simi_values = [s[1] for s in similarities]
    scaler = MinMaxScaler()
    scaled_simi_values = scaler.fit_transform(np.array(simi_values).reshape(-1, 1)).flatten()
    # similarities = [(sentences[i+1], scaled_value * weight[i]) for i, scaled_value in enumerate(scaled_simi_values)]
    similarities = [(sentences[i+1], scaled_value) for i, scaled_value in enumerate(scaled_simi_values)]


    similarities.sort(key=lambda x: x[1], reverse=True)
    top_k = [item[0] for item in similarities[:1]]
    simi = [item[1] for item in similarities[:1]]
    top_5 = [item[0] for item in similarities[:5]]
    simi_5 = [item[1] for item in similarities[:5]]
    return top_5, simi_5

## Evidence Retrival XML

### Test data

In [ ]:
list_evidence_top5 = []
list_evidence_top1 = []

for i in tqdm(range(len(test_data))):
    statement = test_data.Statement[i]
    context = test_data.splited_sentences[i]

    evidence_top5, top5_consine = similarities(context, statement) # top 5

    list_evidence_top5.append(evidence_top5)

test_data['evidence_top5'] = list_evidence_top5

  0%|          | 0/1447 [00:00<?, ?it/s]

In [ ]:
list_evidence_top5 = []
list_evidence_top1 = []

for i in tqdm(range(len(dev_data))):
    statement = dev_data.Statement[i]
    context = dev_data.splited_sentences[i]

    evidence_top5, top5_consine = similarities(context, statement) # top 5

    list_evidence_top5.append(evidence_top5)

dev_data['evidence_top5'] = list_evidence_top5

  0%|          | 0/723 [00:00<?, ?it/s]

In [ ]:
# train_data.to_csv('train_sbert.csv', index=False)
# test_data.to_csv('test_sbert.csv', index=False)
# dev_data.to_csv('dev_sbert.csv', index=False)

In [ ]:
def acc_at_k(retrieved, gold, k):
    top_k_retrieved = retrieved[:k]

    remaining_gold = list(gold)
    remaining_retrieved = list(top_k_retrieved)

    correct_retrievals = 0

    gold_indices_to_remove = set()
    retrieved_indices_to_remove = set()

    for gold_idx, gold_evidence in enumerate(remaining_gold):
        for retrieved_idx, retrieved_evidence in enumerate(remaining_retrieved):
            if gold_evidence in retrieved_evidence or retrieved_evidence in gold_evidence:
                correct_retrievals += 1
                gold_indices_to_remove.add(gold_idx)
                retrieved_indices_to_remove.add(retrieved_idx)
                break

    remaining_gold = [e for i, e in enumerate(remaining_gold) if i not in gold_indices_to_remove]
    remaining_retrieved = [e for i, e in enumerate(remaining_retrieved) if i not in retrieved_indices_to_remove]

    acc_k = correct_retrievals / len(gold) if gold else 0

    return acc_k

i = 12
retrieved_evidence = test_data.evidence_top5[i]
gold_evidence = test_data.Evidence_List[i]

k = 5
acc_k_result = acc_at_k(retrieved_evidence, gold_evidence, k)
print(f"ACC@{k}: {acc_k_result}")

ACC@5: 1.0


## ACC@K

In [ ]:
import numpy as np

results = {
    "all": {"all": [], 0: [], 1: [], 2: []},
    "multi": {"all": [], 0: [], 1: [], 2: []},
    "single": {"all": [], 0: [], 1: [], 2: []}
}

for i in range(len(test_data)):
    retrieved_evidence = test_data.evidence_top5[i]
    gold_evidence = test_data.Evidence_List[i]
    label = test_data.labels[i]

    acc_k_result = acc_at_k(retrieved_evidence, gold_evidence, 10)

    results["all"]["all"].append(acc_k_result)
    results["all"][label].append(acc_k_result)

    if len(gold_evidence) > 1:
        results["multi"]["all"].append(acc_k_result)
        results["multi"][label].append(acc_k_result)
    elif len(gold_evidence) == 1:
        results["single"]["all"].append(acc_k_result)
        results["single"][label].append(acc_k_result)

def calculate_mean(values):
    return round(np.mean(values), 4) if values else 0

for case in ["all", "multi", "single"]:
    print(f"acc_k_result {case}:")
    print(f"overall: {calculate_mean(results[case]['all'])}")
    for label in [0, 1, 2]:
        print(f"{label}: {calculate_mean(results[case][label])}")
    print()

acc_k_result all:
overall: 0.5776
0: 0.555
1: 0.5778
2: 0.6019

acc_k_result multi:
overall: 0.5747
0: 0.5703
1: 0.5801
2: 0.5737

acc_k_result single:
overall: 0.5789
0: 0.5468
1: 0.5765
2: 0.609



## mAP@K

In [ ]:
def map_at_k(retrieved, gold, k):
    top_k_retrieved = retrieved[:k]

    # Initialize variables for calculation
    num_relevant = 0
    precision_at_i_sum = 0.0

    # Track which gold items have been retrieved
    retrieved_gold_indices = set()

    for i, retrieved_evidence in enumerate(top_k_retrieved):
        for gold_idx, gold_evidence in enumerate(gold):
            if gold_idx in retrieved_gold_indices:
                continue

            if gold_evidence in retrieved_evidence or retrieved_evidence in gold_evidence:
                num_relevant += 1
                retrieved_gold_indices.add(gold_idx)
                precision_at_i_sum += num_relevant / (i + 1)
                break

    # Calculate mAP@K
    map_k = precision_at_i_sum / len(gold) if gold else 0

    return map_k

# Example usage
retrieved = ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']
gold = ['doc1', 'doc3', 'doc5']
k = 5

map_k = map_at_k(retrieved, gold, k)
print(f"mAP@{k}: {map_k}")

mAP@5: 0.7555555555555555


In [ ]:
import numpy as np

results = {
    "all": {"all": [], 0: [], 1: [], 2: []},
    "multi": {"all": [], 0: [], 1: [], 2: []},
    "single": {"all": [], 0: [], 1: [], 2: []}
}

for i in range(len(test_data)):
    retrieved_evidence = test_data.evidence_top5[i]
    gold_evidence = test_data.Evidence_List[i]
    label = test_data.labels[i]

    map_k_result = map_at_k(retrieved_evidence, gold_evidence, 5)

    results["all"]["all"].append(map_k_result)
    results["all"][label].append(map_k_result)

    if len(gold_evidence) > 1:
        results["multi"]["all"].append(map_k_result)
        results["multi"][label].append(map_k_result)
    elif len(gold_evidence) == 1:
        results["single"]["all"].append(map_k_result)
        results["single"][label].append(map_k_result)

def calculate_mean(values):
    return round(np.mean(values), 4) if values else 0

for case in ["all", "multi", "single"]:
    print(f"mAP_k_result {case}:")
    print(f"overall: {calculate_mean(results[case]['all'])}")
    for label in [0, 1, 2]:
        print(f"{label}: {calculate_mean(results[case][label])}")
    print()

mAP_k_result all:
overall: 0.5083
0: 0.4918
1: 0.4979
2: 0.5364

mAP_k_result multi:
overall: 0.4703
0: 0.4669
1: 0.4679
2: 0.4805

mAP_k_result single:
overall: 0.5246
0: 0.5051
1: 0.5137
2: 0.5506



In [ ]:
import numpy as np

acc_k_results_all = []
acc_k_results_0 = []
acc_k_results_1 = []
acc_k_results_2 = []

for i in range(len(test_data)):
    retrieved_evidence = test_data.evidence_top5[i]
    gold_evidence = test_data.Evidence_List[i]
    label = test_data.labels[i]

    acc_k_result = acc_at_k(retrieved_evidence, gold_evidence, 5)
    acc_k_results_all.append(acc_k_result)

    if label == 0:
        acc_k_results_0.append(acc_k_result)
    elif label == 1:
        acc_k_results_1.append(acc_k_result)
    elif label == 2:
        acc_k_results_2.append(acc_k_result)

average_acc_k_result_all = np.mean(acc_k_results_all)
average_acc_k_result_0 = np.mean(acc_k_results_0) if acc_k_results_0 else 0
average_acc_k_result_1 = np.mean(acc_k_results_1) if acc_k_results_1 else 0
average_acc_k_result_2 = np.mean(acc_k_results_2) if acc_k_results_2 else 0

print(f"Overall: {average_acc_k_result_all}")
print(f"Support: {average_acc_k_result_0}")
print(f"Refuted: {average_acc_k_result_1}")
print(f"NEI: {average_acc_k_result_2}")

Overall: 0.5776384638167636
Support: 0.555005624296963
Refuted: 0.5777777777777777
NEI: 0.6019108280254777


## F1@K

In [ ]:
def f1_at_k(retrieved, gold, k):
    top_k_retrieved = retrieved[:k]

    # Calculate Precision@K
    num_correct_retrievals = 0
    gold_indices_to_remove = set()

    for retrieved_evidence in top_k_retrieved:
        for gold_idx, gold_evidence in enumerate(gold):
            if gold_idx in gold_indices_to_remove:
                continue

            if gold_evidence in retrieved_evidence or retrieved_evidence in gold_evidence:
                num_correct_retrievals += 1
                gold_indices_to_remove.add(gold_idx)
                break

    precision_k = num_correct_retrievals / k if k > 0 else 0

    # Calculate Recall@K
    recall_k = num_correct_retrievals / len(gold) if gold else 0

    # Calculate F1@K
    if precision_k + recall_k == 0:
        f1_k = 0
    else:
        f1_k = 2 * (precision_k * recall_k) / (precision_k + recall_k)

    return f1_k

# Example usage
retrieved = ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']
gold = ['doc1', 'doc3', 'doc5']
k = 5

f1_k = f1_at_k(retrieved, gold, k)
print(f"F1@{k}: {f1_k}")

F1@5: 0.7499999999999999


In [ ]:
import numpy as np

results = {
    "all": {"all": [], 0: [], 1: [], 2: []},
    "multi": {"all": [], 0: [], 1: [], 2: []},
    "single": {"all": [], 0: [], 1: [], 2: []}
}

for i in range(len(test_data)):
    retrieved_evidence = test_data.evidence_top5[i]
    gold_evidence = test_data.Evidence_List[i]
    label = test_data.labels[i]

    f1_k_result = f1_at_k(retrieved_evidence, gold_evidence, 5)

    results["all"]["all"].append(f1_k_result)
    results["all"][label].append(f1_k_result)

    if len(gold_evidence) > 1:
        results["multi"]["all"].append(f1_k_result)
        results["multi"][label].append(f1_k_result)
    elif len(gold_evidence) == 1:
        results["single"]["all"].append(f1_k_result)
        results["single"][label].append(f1_k_result)

def calculate_mean(values):
    return round(np.mean(values), 4) if values else 0

for case in ["all", "multi", "single"]:
    print(f"mAP_k_result {case}:")
    print(f"overall: {calculate_mean(results[case]['all'])}")
    for label in [0, 1, 2]:
        print(f"{label}: {calculate_mean(results[case][label])}")
    print()

mAP_k_result all:
overall: 0.2395
0: 0.242
1: 0.2459
2: 0.2305

mAP_k_result multi:
overall: 0.3486
0: 0.3538
1: 0.3483
2: 0.3392

mAP_k_result single:
overall: 0.193
0: 0.1823
1: 0.1922
2: 0.203

